# Parameter efficient fine-tuning

В рамках семинаров мы ограничены одной не очень большой GPU доступной на колабе (t4 с 16 гб памяти). Поэтому в предыдущих семинарах, когда нужно было что-то зафайнтюнить мы использовали самые маленькие языковые модели (opt-125m, например), иначе мы бы столкнулись с OOM ошибкой или слишком долгим обучением. Естественно качество таких моделей не впечатляет и хотелось бы попробовать модели побольше. Даже сильно побольше, так как кажется, что [эмержентные](https://ru.wikipedia.org/wiki/%D0%AD%D0%BC%D0%B5%D1%80%D0%B4%D0%B6%D0%B5%D0%BD%D1%82%D0%BD%D0%BE%D1%81%D1%82%D1%8C)
свойства, о которых все сейчас говорят, начинают проявлятся у моделей размером около 6-10 миллиардов параметров (https://arxiv.org/pdf/2206.07682.pdf). 
По умолчанию модель opt-6.7b требует около 25 гб видеопамяти, то есть даже для инференса ресурсов колаба не хватит, не говоря даже о обучении (для него понадобится в 4 раза больше).

К счастью нехватка ресурсов - общая проблема. Даже те, у кого есть такие ресурсы заинтересованы в оптимизации (можно использовать меньше ресурсов=денег или же использовать такое же количество ресурсов, но обслуживать больше пользователей=зарабатывать больше денег). Поэтому усилия многих исследователей и компаний направлены в сторону оптимизации больших языковых моделей.


В этом семинаре мы разбере несколько уже разработанных подходов и сможем обучить модель ~~facebook/opt-6.7b в колабе~~ (на самом деле получится только opt-1.3b, так как все библиотеки еще новые и нестабильные)!


Оптимизация базовой модели и оптимизация процесса дообучения это немного разные вещи, поэтому разберем их по очереди.



## Оптимизация предобученных моделей
Для уменьшения готовой модели есть два основных подхода: дистиляция и квантизация. 

**Дистиляция** (knowledge distillation) - это обучение меньшей по размеру модели воспроизводить предсказания большей модели. Исходная модель при таком подходе называется учителем, а меньшая модель - учеником. Например, можно взять большой предобученный классификатор тональности, сделать предсказания на каком-нибудь большом корпусе и обучить меньшую модель на этих предсказаниях. Так как при уменьшении количества параметров будет уменьшаться и качество, можно ограничить обучающий корпус каким-то одним доменом, чтобы упростить меньшей модели задачу (меньшая модель, например, научится хорошо определять тональность коротких текстов, но будет плохо работать с длинными; на практике это может быть приемлимый трейдофф)

Схема дистилляции:
![](https://miro.medium.com/v2/resize:fit:936/1*8KqNtABnNXM527JK9UuBUQ.jpeg)

Применять дистиляцию к простым языковым моделям нет особого смысла, так как обучающие данные и так доступны и можно просто обучить меньшую модель с нуля аналогично большой модели. Но уместна дистиляция для моделей, обученных на инструкциях. Вот например эксперименты по дистиляции GPT в небольшие модели - https://github.com/mbzuai-nlp/LaMini-LM В целом это очень похоже на [Alpaca](https://github.com/tatsu-lab/stanford_alpaca), но датасет здесь намного больше, а сама модель обучается с нуля (в Alpaca дообучается Llama). И также как и с Alpaca, это не очень то легально - по сути это попытка скопировать модель без доступа к обучающим данным, поэтому лицензия на моделях запрещает коммерческое использование. 


**Квантизация** - это уменьшение размера модели за счет уменьшения точности представления чисел. Веса в модели это просто очень много чисел вида 0.23123125, -1.234559 и для хранения каждого такого числа требуется какое-то количество памяти. Интервал допустимых значений и количество знаков после запятой всегда ограничены, но по умолчанию они достаточно большие и, выясняется, что можно достаточно сильно округлить веса, и при этом, практически не потерять в качестве! 
Схема квантизации: 
![](https://developer-blogs.nvidia.com/wp-content/uploads/2021/07/qat-training-precision.png)

Разумеется, квантизация сложнее, чем просто округление, но подробно методы квантизации мы разбирать не будем. Если вам интересно, можно почитать вот это - https://huggingface.co/blog/hf-bitsandbytes-integration Один из авторов - Tim Dettmers, автор библиотеки bitsandbytes, в которой реализованы методы квантизации и которая постепенно интегрируется в huggingface. 

Давайте посмотрим как использовать квантизацию в huggingface с предобученными моделями.

In [6]:
!pip install -q git+https://github.com/huggingface/transformers.git@main
!pip install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 18.2 MB/s eta 0:00:00


Попробуем загрузить большую модель без дополнительных параметров.

In [1]:
from transformers import AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-1.3b', 
                                             cache_dir='./models').to('cuda')

Ошибка по памяти. По умолчанию веса хранятся в fp32. Давайте попробуем fp16 - формат, который требует в два раза меньше памяти.

## FP16

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-1.3b', 
                                             torch_dtype=torch.float16, # указываем fp16
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

Теперь модель загружается и мы можем даже что-то сгенерировать

In [3]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, no_repeat_ngram_size=2)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
I'm not sure if you're being sarcastic or not, but I'm going to go with sarcastic.


## 8-bit

Можно пойти еще дальше и попробовать 8-битный формат, который требует в 4 раза меньше памяти. Так мы можем попробовать даже еще большую модель

In [3]:
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.4 MB/s eta 0:00:00


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-2.7b', 
                                             load_in_8bit=True, 
                                             device_map='auto',
                                             cache_dir='./models')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-2.7b")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, no_repeat_ngram_size=2)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 In the beginning the Universe was created.   This has made a lot of people very angry and been widely regarded as a bad move.
I'm not sure if I should upvote or downvote this.


## Оптимизация обучения

Квантизация также применима при обучении моделей, но тут появляются дополнительные сложности вроде нестабильности и резкого ухудшения качества (плюс зависимости от железа). Методы решения этих проблем активно разработываются и многое уже работает, но будьте готовы к непонятным ошибкам и неподдерживаемым моделям при использовании квантизации. Согласно вычислениями вот отсюда - https://blog.eleuther.ai/transformer-math/ - при 8bit представлениях можно уместить в колаб и 6.7b модель. Также она используется в оригинальном [туториале](https://colab.research.google.com/drive/1jCkpikz0J2o20FBQmYmAGdiKmJGOMo-o?usp=sharing#scrollTo=cg3fiQOvmI3Q), но у меня не получилось этого сделать. При загрузке 6.7b модели кернел крашится из-за нехватки памяти. 

В любом случае, даже с квантизированной моделью полный файн-тюнинг - это все еще очень дорого. Обучение требует в среднем в 4 раза больше памяти, чем инференс, т.е. модель, которую мы загрузили в 8bit и уместили на 1 гпу, обучать мы не сможем.

К счастью, уже появились методы частичного дообучения. Самый популярный на данный момент - LoRA ( https://arxiv.org/abs/2106.09685 ). Мы уже использовали частичное дообучение на предыдущих занятиях, когда добавляли 1 дополнительный полносвязный слой к предобученной модели и обучали только его. LoRA - это обобщение такого подхода, но тут, дополнительные веса добавляются к каждому полносвязному слою в предобученной модели. На одном из прошлых семинаров мы разбирали, что большую матрицу можно представить как произведение нескольких матриц поменьше. Причем внутреннюю размерность этих матриц можно варьировать - чем меньше, тем хуже восстанавливается оригинальная матрица, но тем меньше памяти занимается. В LoRA используется такой же прием - к каждой матрице с весами (полносвязному слою) прибавляется матрица такого же размера, которая получена произведением двух матриц поменьше A и B. Внутренюю размерность этих матриц можно варьировать - это гиперпараметр модели (см. ниже), чем она больше, тем выше точность модели, и тем выше потребность в памяти.

Схема LoRA:
![](https://miro.medium.com/v2/resize:fit:730/1*D_i25E9dTd_5HMa45zITSg.png)


При небольшой внутренней размерности получается, что дообучение требует лишь небольшой процент ресурсов, требуемых для полноценного обучения. Также полученные веса очень удобно использовать на практике - их можно сохранять и подгружать отдельно, не затрагивая оригинальною большую модель. Предобученные модели весят гигабайты, а LoRA веса - несколько мегабайт. Можно также дообучить несколько LoRA весов и переключаться между ними на лету.

LoRA реализована в отдельной библиотеке PEFT в экосистеме huggingface. Давайте попробуем дообучить большую модель

### Fine-tuning

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 31.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce

В итоге я использую модель facebook/opt-1.3b, так как 6.7b не помещается в колаб, а 2.7b модель обучается некорректно.

In [2]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b", 
    load_in_8bit=True, 
    device_map='auto',
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [3]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

В этой ячейке все веса изначальной модели замораживаются

In [4]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

In [5]:
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [6]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [7]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=32,
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6291456 || all params: 1322049536 || trainable%: 0.47588655558516074


Параметр r отвечает за внутренюю размерность дополнительных матриц. При r=32 обучаться в итоге будет около 3м параметров, что меньше 1 процента от всех параметров изначальное модели. 

### Обучение

In [9]:
import transformers
from datasets import load_dataset

# в качестве датасета используются цитаты на англиском языке
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=400, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=1, 
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.789200
2,2.583300


Сохраним модель (сохранятся только дополнительные веса)

In [ ]:
model.save_pretrained('opt_1.3_lora')

Чтобы загрузить обученную модель нужно сначала загрузить базовую модель, а потом применить к ней LoRa веса

In [22]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "opt_1.3_lora"

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="facebook/opt-1.3b", 
                                             return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [23]:
def generate(text, tokenizer, model):
  batch = tokenizer(text, return_tensors='pt').to('cuda')
  output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.0, no_repeat_ngram_size=2)

  return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

Давайте попробуем сделать несколько предсказаний, используя базовую модель, чтобы потом сравнить с дообученной

In [24]:
generate("I have a dream that",  tokenizer, model)

"I have a dream that one day, the world will be a better place.\nI've got a feeling that's not going to happen."

In [25]:
generate("You know you're in love when",  tokenizer, model)

"You know you're in love when you can't stop thinking about her.\nI'm in a relationship with a girl who is the same way. I can never stop looking at her, and I'm not sure if I should be happy or sad."

In [26]:
generate("I am so clever that",  tokenizer, model)

"I am so clever that I can't even tell if this is a joke or not.\nI'm not sure if you're being sarcastic or if I'm being serious."

Загрузим LoRA веса и попробуем те же промпты

In [29]:
model = PeftModel.from_pretrained(model, peft_model_id)


In [30]:
generate("I have a dream that",  tokenizer, model)

"I have a dream that one day, the world will be a better place.\nI've got a feeling that's not going to happen."

In [31]:
generate("You know you're in love when",  tokenizer, model)

"You know you're in love when you can't stop thinking about her.\nI'm in a relationship with a girl who is the same way. I can never stop looking at her, and I'm not sure if I should be happy or sad."

In [32]:
generate("I am so clever that",  tokenizer, model)

"I am so clever that I can't even tell if this is a joke or not.\nI'm not sure if you're being sarcastic or if I'm being serious."